In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
import matplotlib
from sklearn.metrics import mean_squared_error
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
import lightgbm as lgb
sns.set()

In [ ]:
train=pd.read_csv("../input/X_train.csv")
train.head()

In [ ]:
test=pd.read_csv("../input/X_test.csv")
test.head(10)

In [ ]:
train['series_id'].unique(),test['series_id'].unique()

In [ ]:
sample_submission=pd.read_csv("../input/sample_submission.csv")
sample_submission.head()

In [ ]:
test_id=sample_submission['series_id']

In [ ]:
sample_submission.shape

In [ ]:
y_train=pd.read_csv("../input/y_train.csv")
y_train.head(10)

In [ ]:
y_train['surface'].value_counts().plot.bar();

In [ ]:
train.shape,test.shape,y_train.shape

In [ ]:
plt.figure(figsize=(26, 20))
for i, col in enumerate(train.columns[3:]):
    plt.subplot(3, 4, i + 1)
    plt.plot(train.loc[train['series_id'] == 1, col])
    plt.title(col)

In [ ]:
plt.figure(figsize=(26, 20))
for i, col in enumerate(test.columns[3:]):
    plt.subplot(3, 4, i + 1)
    plt.plot(test.loc[test['series_id'] == 1, col])
    plt.title(col)

In [ ]:
# refrence from https://www.kaggle.com/jsaguiar/surface-recognition-baseline
def feature_extraction(raw_frame):
    frame = pd.DataFrame()
    raw_frame['angular_velocity'] = raw_frame['angular_velocity_X'] + raw_frame['angular_velocity_Y'] + raw_frame['angular_velocity_Z']
    raw_frame['linear_acceleration'] = raw_frame['linear_acceleration_X'] + raw_frame['linear_acceleration_Y'] + raw_frame['linear_acceleration_Y']
    raw_frame['velocity_to_acceleration'] = raw_frame['angular_velocity'] / raw_frame['linear_acceleration']
    
    for col in raw_frame.columns[3:]:
        frame[col + '_mean'] = raw_frame.groupby(['series_id'])[col].mean()
        frame[col + '_std'] = raw_frame.groupby(['series_id'])[col].std()
        frame[col + '_max'] = raw_frame.groupby(['series_id'])[col].max()
        frame[col + '_min'] = raw_frame.groupby(['series_id'])[col].min()
        frame[col + '_max_to_min'] = frame[col + '_max'] / frame[col + '_min']
        
        frame[col + '_mean_abs_change'] = raw_frame.groupby('series_id')[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        frame[col + '_abs_max'] = raw_frame.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
    return frame

In [ ]:
train_df = feature_extraction(train)
test_df = feature_extraction(test)
train_df.head()

In [ ]:
train_df["orientation_X_mean"].hist()

In [ ]:
train_df["orientation_X_std"].hist()

In [ ]:
test_df["orientation_X_mean"].hist()

In [ ]:
test_df["orientation_X_std"].hist()

In [ ]:
train_df.shape,test_df.shape

In [ ]:
le = LabelEncoder()
target_train = le.fit_transform(y_train['surface'])

In [ ]:
train_df['surface']=target_train

In [ ]:
sns.violinplot(data=train_df,x="surface", y="orientation_X_mean")

In [ ]:
sns.violinplot(data=train_df,x="surface", y="orientation_Y_mean")

In [ ]:
sns.violinplot(data=train_df,x="surface", y="orientation_Z_mean")

In [ ]:
sns.violinplot(data=train_df,x="surface", y="angular_velocity_X_mean")

In [ ]:
sns.violinplot(data=train_df,x="surface", y="angular_velocity_Y_mean")

In [ ]:
sns.violinplot(data=train_df,x="surface", y="angular_velocity_Z_mean")

In [ ]:
train_df=train_df.drop(['surface'],axis=1)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(train_df, target_train, random_state = 0)
from sklearn.ensemble import RandomForestClassifier
alg1 =  RandomForestClassifier()
alg1.fit(X_train, Y_train)

In [ ]:
Y_pred = alg1.predict(X_test)
train_score = alg1.score(X_train, Y_train)
test_score = alg1.score(X_test, Y_test)
print("Train Score: ", train_score)
print("Test Score: ", test_score)

In [ ]:
params = {
    'num_leaves': 20,
    'min_data_in_leaf': 15,
    'objective': 'multiclass',
    'max_depth': 10,
    'learning_rate': 0.01,
    "boosting": "gbdt",
    "bagging_freq": 5,
    "bagging_fraction": 0.8126672064208567,
    "bagging_seed": 11,
    "verbosity": -1,
    'reg_alpha': 0.1302650970728192,
    'reg_lambda': 0.3603427518866501,
    "num_class": 9,
    'nthread': -1
}

def multiclass_accuracy(preds, train_data):
    labels = train_data.get_label()
    pred_class = np.argmax(preds.reshape(9, -1).T, axis=1)
    return 'multi_accuracy', np.mean(labels == pred_class), True

t0 = time.time()
train_set = lgb.Dataset(train_df, label=target_train)
eval_hist = lgb.cv(params, train_set, nfold=8, num_boost_round=1400,
                   early_stopping_rounds=80, seed=19, feval=multiclass_accuracy)
num_rounds = len(eval_hist['multi_logloss-mean'])
# retrain the model and make predictions for test set
clf = lgb.train(params, train_set, num_boost_round=num_rounds)
predictions = clf.predict(test_df, num_iteration=None)
print("Timer: {:.1f}s".format(time.time() - t0))

In [ ]:
predictions

In [ ]:
v1, v2 = eval_hist['multi_logloss-mean'][-1], eval_hist['multi_accuracy-mean'][-1]
print("Validation logloss: {:.4f}, accuracy: {:.4f}".format(v1, v2))
plt.figure(figsize=(10, 4))
plt.title("CV multiclass logloss")
num_rounds = len(eval_hist['multi_logloss-mean'])
ax = sns.lineplot(x=range(num_rounds), y=eval_hist['multi_logloss-mean'])
ax2 = ax.twinx()
p = sns.lineplot(x=range(num_rounds), y=eval_hist['multi_logloss-stdv'], ax=ax2, color='r')

plt.figure(figsize=(10, 4))
plt.title("CV multiclass accuracy")
num_rounds = len(eval_hist['multi_accuracy-mean'])
ax = sns.lineplot(x=range(num_rounds), y=eval_hist['multi_accuracy-mean'])
ax2 = ax.twinx()
p = sns.lineplot(x=range(num_rounds), y=eval_hist['multi_accuracy-stdv'], ax=ax2, color='r')

In [ ]:
importance = pd.DataFrame({'gain': clf.feature_importance(importance_type='gain'),
                           'feature': clf.feature_name()})
importance.sort_values(by='gain', ascending=False, inplace=True)
plt.figure(figsize=(10, 20))
ax = sns.barplot(x='gain', y='feature', data=importance)

In [ ]:
sample_submission['surface'] = le.inverse_transform(predictions.argmax(axis=1))
sample_submission.to_csv('Lgb.csv', index=False)